# Setup

In [13]:
import os
import getpass
import json

from dotenv import load_dotenv

from IPython.display import display, Markdown, clear_output

In [14]:
def display_md(md):
    clear_output()
    display(Markdown(md))

def display_json(json_data):
    display_md(f'```json\n{json.dumps(json_data, indent=2)}\n```')

In [2]:
# Set environment variable if not already set
def _set_env(var: str):
    if not os.environ.get(var):
        os.environ[var] = getpass.getpass(f"{var}")

In [3]:
# Load environment variables from .env file
load_dotenv()

# API keys to check
api_list = ["GROQ_API_KEY", "TAVILY_API_KEY", "LANGCHAIN_API_KEY", "OPENAI_API_KEY", "LANGCHAIN_TRACING_V2", "LANGCHAIN_ENDPOINT", "LANGCHAIN_PROJECT"]

# Check if API keys are set
for api in api_list:
    if os.getenv(api) is None:
        print(f"{api} is not set")
        _set_env(api)
    else:
        print(f"{api} is set")

GROQ_API_KEY is set
TAVILY_API_KEY is set
LANGCHAIN_API_KEY is set
OPENAI_API_KEY is set
LANGCHAIN_TRACING_V2 is set
LANGCHAIN_ENDPOINT is set
LANGCHAIN_PROJECT is set


# Create the LangChain agent

In [4]:
from langchain import hub
from langchain.agents import create_openai_functions_agent
from langchain_openai.chat_models import ChatOpenAI
from langchain_community.tools.tavily_search import TavilySearchResults

In [5]:
tools = [TavilySearchResults(max_results=1)]

In [18]:
# Get the prompt to use - you can modify this!
prompt = hub.pull("hwchase17/openai-functions-agent")
display_json(prompt.dict())

```json
{
  "name": null,
  "input_variables": [
    "agent_scratchpad",
    "input"
  ],
  "optional_variables": [
    "chat_history"
  ],
  "output_parser": null,
  "partial_variables": {
    "chat_history": []
  },
  "metadata": {
    "lc_hub_owner": "hwchase17",
    "lc_hub_repo": "openai-functions-agent",
    "lc_hub_commit_hash": "a1655024b06afbd95d17449f21316291e0726f13dcfaf990cc0d18087ad689a5"
  },
  "tags": null,
  "messages": [
    {},
    {},
    {},
    {}
  ],
  "validate_template": false,
  "_type": "chat"
}
```

In [7]:
# Choose the LLM that will drive the agent
llm = ChatOpenAI(model="gpt-4o-mini", streaming=True)

In [9]:
# Construct the OpenAI Functions agent
agent_runnable = create_openai_functions_agent(llm, tools, prompt)

## Define the graph state

We now define the graph state. The state for the traditional LangChain agent has a few attributes

1. `input`: This is the input string representing the main ask from the user, passed in as input.
2. `chat_history`: This is any previous coversation messages, also passed in as input.
3. `intermediate_steps`: This is list of actions and corresponding observations that the agent takes over time. This is updated each iteration of the agent.
4. `agent_outcome`: This is the response from the agent, either and AgentAction or AgentFinish. the AgentExecutor should finish when this is an AgentFinish, otherwise it should call the requested tools.